**Содержание**<a id='toc0_'></a>    
- [Самый обычный градиентный спуск](#toc1_)    
- [Модификации градиентного спуска](#toc2_)    
    - [Cтохастический градиентный спуск с импульсом](#toc2_1_1_)    
    - [Cтохастический градиентный спуск с импульсом с заглядыванием вперед](#toc2_1_2_)    
    - [Похожий вариант](#toc2_1_3_)    
- [Некоторые другие методы оптимизации](#toc3_)    
    - [Rprop](#toc3_1_1_)    
    - [RMSprop](#toc3_1_2_)    
    - [Adam](#toc3_1_3_)    
- [Теоретические задачи: Понимаем SGD с momentum](#toc4_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

# <a id='toc1_'></a>[Самый обычный градиентный спуск](#toc0_)

$$\nabla f = [\frac{\partial f}{\partial w_0}, ... ,\frac{\partial f}{\partial w_n}]^T$$
$$w^1 = w^0 - \alpha \nabla f(w^0) \\
w^2 = w^1 - \alpha \nabla f(w^1) \\
... \\
w^{t+1} = w^t - \alpha \nabla f(w^t)$$

"Истинный" градиент функции потерь должен вычисляться по потерям (средним, суммарным или еще как агрегированным) на всем объеме данных.

Если вместо этого шаги делаются по градиенту потерь, расчитанному на **одном** примере, то такой метод называется 
**стохастическим градиентным спуском**.

Если для расчета градиента функции потерь используется случайное подмножество исходных данных, то такой метод называется **мини-пакетным градиентным спуском** (по сути разновидность SGD, во многих пакетах для них используется одно название - SGD)

Основное правило по выбору размера батча: 
- $ 1 \ll |b| \ll N$

# <a id='toc2_'></a>[Модификации градиентного спуска](#toc0_)

Основная проблема SGD - области "овражного" типа.

### <a id='toc2_1_1_'></a>[Cтохастический градиентный спуск с импульсом](#toc0_)

На помощь приходят аналогии из физики - катящийся шар (добавление инреции)

$$ \begin{cases} 
\frac{\partial v}{\partial t} = \frac{1}{m} (F + F_{тр}) = \frac{1}{m} \nabla f - \frac{1}{m} \eta v\\
\frac{\partial x}{\partial t} = v
\end{cases}$$

- уравнения движения материальной точки (2-й з-н Ньютона: ускорение сила/массу, определение скорости)
  - $F$ - антиградиент функции потерь (потенциальная сила)
  - $F_{тр}$ - сила вязкого трения
  - $\eta$ - к-т вязкого трения

Методом конечных разностей:

$\frac{\partial f}{\partial x} = \lim_{\Delta x \to 0} \frac{f(x - \Delta x) - f(x)}{\Delta x} \approx \frac{f(x - \Delta x) - f(x)}{\Delta x}$ - слева определение производной, справа - конечная разность, деленная на шаг

\*конечная разность при фиксированном шаге есть линейный оператор, отображающий пространство непрерывных функций в себя   
\*\*разделённая разность — обобщение понятия производной для дискретного набора точек

$$\begin{cases} 
\frac{\partial v}{\partial t} = \frac{v^{t+1} - v^{t}}{\Delta t}\\
\frac{\partial x}{\partial t} = \frac{x^{t+1} - x^{t}}{\Delta t}
\end{cases}$$

Подставим это в исходную систему уравнений и выразим $v^{t+1}, x^{t+1}$

$$\begin{cases} 
v^{t+1}= v^t - \frac{\Delta t}{m} \nabla f - \frac{\Delta t}{m} \eta v^t \\
x^{t+1} = x^t + v^t \Delta t
\end{cases}$$

С учетом, что координата это у нас веса, обозначив $\alpha := \Delta t $ (скорость обучения - это дискретность времени), $\beta := 1 - \frac{\Delta t}{m} \eta$, и приняв масштаб $\frac{\Delta t}{m} = 1$, получим:


$$\begin{cases} 
w^{t+1} = w^t + \alpha v^t \\
v^{t+1}= v^t \beta - \nabla f
\end{cases}$$

Это называется **стохастический градиентный спуск с импульсом** (моментом). Параметры: скорость обучения $\alpha$ и к-т импульса $\beta$

\*) получается "скорость" - параметр, где градиент "накапливается" (когда двигаемся вниз) и "расходуется" (если пришлось двинуться вверх). Снижение "скорости" замедляет движение вдоль того направления, где мы "влезаем на стену оврага" и не трогает те направления, где мы движемся к минимуму. Ну логично, все как в жизни...

### <a id='toc2_1_2_'></a>[Cтохастический градиентный спуск с импульсом с заглядыванием вперед](#toc0_)

$$ \begin{cases} 
\frac{\partial v}{\partial t} = \frac{1}{m} (F + F_{тр}) = \frac{1}{m} \nabla f - \frac{1}{m} \eta v\\
\frac{\partial x}{\partial t} = v
\end{cases}$$

...

$$\begin{cases} 
w^{t+1} = w^t + \alpha v^t \\
v^{t+1}= v^t \beta - \nabla f (w^t + \alpha v^t)
\end{cases}$$

- что-то вроде дополнительной инерции, которая дополнительно уменьшает градиент если он пошел "в гору", но и дополнительно ускоряет, если идет вниз (?)... не понятно до конца

### <a id='toc2_1_3_'></a>[Похожий вариант](#toc0_)

$$w^{t+1} = w^t - \alpha EMA (\nabla f)^t $$

- EMA - экспоненциальное скользящее среднее градиента лосс-функции в момент времени t:

$$EMA_{\beta}(\nabla f)^t = (1 -\beta) \nabla f^t + \beta \cdot EMA_{\beta}(\nabla f)^{t-1}$$

- это такая "функция с памятью", она сглаживает последние несколько значений градиента (в данном примере – только текущее и предыдущее, логично инициализировать его нулём)

Почти тоже, что обычный стохастический градиентный спуск с испульсом. Почему? В стохастическом градиентном спуске с испульсом у нас к скорости прибавлялись градиенты лосс-функции, а здесь у нас к координате прибавляются усреднённые градиенты лосс-функции, то есть раньше усреднение происходило в параметре скорости, а теперь усреднение происходит в экспоненциальном скользящем среднем. 


# <a id='toc3_'></a>[Некоторые другие методы оптимизации](#toc0_)

### <a id='toc3_1_1_'></a>[Rprop](#toc0_)

В SGD и рассмотренных модификация одинаковые lr для всех параметров. Сделаем индивинуальный lr для каждого!

Будем учитывать только знаки градиента: $sign (\nabla f)$

Возьмем начальный lr: $\alpha_0$

$$w_i^{t+1} = w_i^t - \alpha_i^t \cdot sign (\nabla f_i(w^t))$$

- по тем параметрам, по которым градиент растет, шаг будет сделан в противоположном направлении
- сама величина тоже меняется
$$\alpha_i^{t+1} = \begin{cases} 
\alpha_i^t \cdot 1.2 & sign (\nabla f_i(w^t) \cdot \nabla f_i(w^{t-1}) ) > 0 \\
\alpha_i^t \cdot 0.6 & sign (\nabla f_i(w^t) \cdot \nabla f_i(w^{t-1}) ) \leq 0 
\end{cases}$$

- если знак градиента не менялся на последнем шаге, можно чуть ускорить спуск по этому параметру
- если изменился или стоит на месте - сильно уменьшить шаг

**Недостаток**: плохо работает с батчами, оптимизацию надо проводить для каждого элемента отдельно, при этом сильно эмпирические коэффициенты

### <a id='toc3_1_2_'></a>[RMSprop](#toc0_)

Root Mean Square Propagation. Сильно похож на Rprop

$$w_i^{t+1} = w_i^t - \alpha \frac{\nabla f(w^t)}{\sqrt {EMA_{\gamma}(\nabla f^2)^t}}$$

- Интуиция этого алгоритма заключается в том, что если вдоль какого-то направления производная слишком большая, это означает что вдоль этого направления нужно идти чуть-чуть помедленнее, потому что очень уж быстро изменяется функция потерь по этому направлению. А если вдоль какого-то другого параметра функция потерь слишком маленькая, то вдоль этого параметра нужно идти чуть побыстрее, потому что функция потерь вдоль этого параметра изменяется слишком медленно. Таким образом у нас подстраивается скорость обучения под каждый из параметров.
- а средний шаг получится $\alpha$

### <a id='toc3_1_3_'></a>[Adam](#toc0_)

$$w_i^{t+1} = w_i^t - \alpha \frac{EMA_{\beta_1}(\nabla f)^t}{\sqrt {EMA_{\beta_2}(\nabla f^2)^t} + \varepsilon}$$

- объединяет как положительные качества стохастического градиентного спуска с моментом (импульсом), и метода RMSProp
- $\varepsilon$ чтоб на 0 не поделить ненароком. Иногда его под корень заводят в заквадраченном виде, не суть
- умолчания в торче и других пакетах $\alpha= 3 \cdot 10^{-4}$, параметр экспоненциального скользящего среднего в числителе - $\beta_1=0.9$, $\beta_2= 0.999$.
  - Бывают ситуации, когда некоторые нейронные сети лучше учатся с какими-то другими параметрами алгоритма Adam. Однако для очень большого класса нейронных сетей вот такие параметры показывают очень хорошие результаты. 
 
Если вы хотите что-то быстро проверить и как-то обучить нейронную сеть, то вы можете взять алгоритм Adam с вот этими параметрами, и у вас получится какой-то хороший результат. 

Если же вы хотите сделать какую-то очень хорошо оптимизированную под текущую задачу нейронную сеть, то обычно берут стохастический градиентный спуск, и уже для стохастического градиентного спуска кропотливо подбирают необходимый learning rate, для того, чтобы стохастический градиентный спуск хорошо решал задачу.

# <a id='toc4_'></a>[Теоретические задачи: Понимаем SGD с momentum](#toc0_)

Материальная точка массы mm находится в поле потенциальных сил, которые заданы потенциалом $L(\vec{x})$ (в momentum SGD в качестве потенциала берут лосс-функцию, которую нужно минимизировать). Потенциальная сила, которая действует на материальную точку, равна

$$\vec{F}_c = - \nabla L(\vec{x})$$

Среда, в которой движется материальная точка, вязкая. Это означает, что, если материальная точка движется со скоростью $\vec{v}$, то сила трения, которая действует на материальную точку со стороны среды, равна $\vec{F}_f = - \eta \vec{v}$, где $\eta$ - коэффициент вязкого трения. Принимая во внимание Второй Закон Ньютона:

$\vec{F} = m \vec{a}$

найдите ускорение материальной точки, если известна скорость $\vec{v}$ и положение материальной точки $\vec{x}$.

$$\vec a = \frac {-\nabla L(\vec{x}) - \eta  v}{m}$$

Принимая во внимание то, что скорость - это производная координат материальной точки по времени, а ускорение - это производная скорости по времени:
$$\vec{v} = \frac{\partial \vec{x}}{\partial t}$$
$$\vec{a} = \frac{\partial \vec{v}}{\partial t}$$

перепишите выражение для ускорения из предыдущей задачи таким образом, чтобы в правой части остался 0 - это уравнение движения материальной точки. Перепишите его используя только координаты материальной точки (выразите ускорение и скорость через производные координаты) и их производные по времени.

$$\vec a = \frac {-\nabla L(\vec{x}) - \eta  \frac{\partial \vec{x}}{\partial t}}{m} =\frac{\partial \vec{v}}{\partial t} = \frac{\partial \vec{x}}{\partial t^2} \\
\frac{\partial \vec{x}}{\partial t^2} + \frac {1}{m} \nabla L(\vec{x}) + \frac {1}{m} \eta \frac{\partial \vec{x}}{\partial t} = 0$$

Как известно, энергия замкнутой системы остается постоянной. Допустим, что материальная точка начала движение с нулевой скоростью $\vec{v}_0 = 0$, а потенциальная энергия в начальный момент времени была равна $L(x_0)$, а коэффициент трения $\eta = 0$. В любой момент времени, согласно Закону Сохранения Энергии, выполняется равенство:
$$\frac{m v_t^2}{2} + L(x_t) = \frac{m v_0^2}{2} + L(x_0)$$

Какова будет потенциальная энергия материальной точки, когда в следующий раз она полностью остановится?

$$E = L(x_0)$$



В случае, если в системе присутствует сила трения $\eta$ (`eta`), энергия системы не сохраняется, а убывает. Запишите условие, наложенное на `eta` при котором рано или поздно материальная точка застрянет в некотором локальном минимуме потенциальной энергии $L$.

$$\eta > 0$$

Дифференциальные уравнения можно решать численно. Для этого дифференциальные уравнения высоких порядков представляют в виде системы дифференциальных уравнений первого порядка. Наше уравнение можно представить следующим образом:

$$\begin{cases} \frac{\partial \vec{x}}{\partial t} = \vec{v} \\ \frac{\partial \vec{v}}{\partial t} = - \frac{1}{m}\left( \nabla L(\vec{x}) + \eta \vec{v} \right) \end{cases}$$
 
Теперь для решения такого уравнения применяются конечные разности. Приближением производной конечными разностями называется например такое приближение производной:

$$\frac{\partial f}{\partial t} = \lim_{\delta t \rightarrow 0} \frac{f_{t + \delta t} - f_{t}}{\delta t} \approx \frac{f_{t + \Delta t} - f_{t}}{\Delta t}$$
 
при каком-то небольшом (но при этом конечном) шаге по времени $\Delta t$. Стоит отметить, что конечные разности применяются не только для производных по времени, а могут применяться для любых производных. Также стоит отметить, что это не единственный способ приблизить производную функции при помощи конечных разностей.

Допустим, в некоторый момент времени мы знаем координату и скорость материальной точки $\vec{x}_t$, $\vec{v}_t$. При помощи конечных разностей найдите выражение для скорости в следующий момент времени при условии, что правые части уравнений вычисляются из скоростей и координат в текущий момент времени.

$$\begin{cases} v^{t+1}= v^t - \frac{\Delta t}{m} \nabla L(\vec{x}) - \frac{\Delta t}{m} \eta v^t \\
x^{t+1} = x^t + v^t \Delta t \end{cases}$$

Допустим, что точка $x_0$, с которой мы начинаем оптимизацию, отстоит от точки оптимума на расстояние $\rho$. Для оптимизации мы будем использовать алгоритм:

$x_{t + 1} = x_t - \alpha \frac{\nabla L}{\sqrt{\|\nabla L\|^2_2 + \epsilon ^ 2}}$, где $\epsilon \ll \|\nabla L\|_2$.

Найдите минимальное число шагов, которое нужно будет сделать таким алгоритмом, чтобы достичь оптимума.

$$steps = \rho / \alpha \frac{\nabla L}{\sqrt{\|\nabla L\|^2_2 + \epsilon ^ 2}} \approx \rho / \alpha$$

Взвешенное среднее некоторой величины ff оценивается следующим образом:
$\bar{f} = \frac{\sum_{i = 1}^N f_i w_i}{\sum_{i = 1}^N w_i}$, где $w_i$ - веса, с которыми взвешиваются значения $f_i$.

Допустим, что мы отслеживаем экспоненциальное скользящее среднее:

$$EMA_\alpha f_{t + 1} = \alpha EMA_\alpha f_{t} + (1 - \alpha) f_{t + 1},$$

причем в начальный момент времени $EMA_\alpha f_0 = 0$.

Если внимательно проанализировать эту процедуру, то можно понять, что экспоненциальное скользящее среднее не является взвешенным средним, а стремится к нему только при $t \to \infty$.

Это особенно хорошо видно при $t = 1$. Допустим, $f_1 = 100$, $\alpha = 0.9$. Тогда $EMA_1 f = 10$, что далеко от взвешенного среднего, которое для одного примера должно равняться $\bar{f} = 100$ (поскольку наблюдался всего один пример и он был равен 100, каким бы вес ни был, взвешенное среднее должно быть равно 100).

На какое значение должно быть разделено экспоненциальное среднее $EMA_\alpha f_t$, чтобы получилось взвешенное среднее?

После получения правильного ответа, подумайте, как нужно поправить алгоритм Adam:

$$x_{t + 1} = x_t - \alpha \frac{EMA_{\beta_1} \nabla L_t}{\sqrt{EMA_{\beta_2} (\nabla L) ^ 2 _t + \epsilon}}$$
чтобы в числителе и знаменателе были правильные оценки взвешенных средних на первых итерациях.

**Если поделить на $1 - alpha^t$, то расписав рекуррентную формулу $EMA$ легко увидеть, что все "забытые" слагаемые будут "вспомнены" в той степени, в какой они к моменту $t$ "забылись"**

**Семинар** про классификацию рукописных цифр в `./Neural_Networks_and_CV`